<font color="white">.</font> | <font color="white">.</font> | <font color="white">.</font>
-- | -- | --
![NASA](http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg) | <h1><font size="+3">ASTG Python Courses</font></h1> | ![NASA](https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png)
 
---

<CENTER>
<H1 style="color:red">
Code Optimization Techniques
</H1>
</CENTER>

In [ ]:
from __future__ import print_function

## <font color='red'>What will be Covered?</font>

* Code Profiling
* Import Statement Overhead
* Functions
* Loop Optimization
* String Manipulation
* Numerical Code

## <font color='red'>Reference Documents</font>

<OL>
<LI> <A HREF="https://www.techbeamers.com/python-code-optimization-tips-tricks/">Essential Python Code Optimization Tips and Tricks</A>
<LI> <A HREF="https://wiki.python.org/moin/PythonSpeed/PerformanceTips">Python Speed/Performance Tips</A>
<LI> <A HREF="https://www.codementor.io/satwikkansal/python-practices-for-efficient-code-performance-memory-and-usability-aze6oiq65">Python Practices for Efficient Code: Performance, Memory, and Usability</A>
</OL>

## <font color='red'>Introduction</font>

We want to write Python codes that have the following features:

* Can run and produce the expected results
* Easy to maintain
* Readable
* Reusable
* Shareable
* **<font color='blue'>Run fast</font>**

At least, when you write a Python code:

* Write simple functions that do as little work as possible.
* Follow the Python coding standards
* Properly document your code
* Write your code as a package even if you do not plan to deploy it 

* Computing resources are expensive and limited.
* Optimization is important in reducing operational costs in terms of storage, memory, or computing power.

**Here, we are interested in identifying techniques to speed up Python codes and reduce the memory footprint.**

## <font color='red'>Profile Your Code </font>

* Before we can optimize our code, it has to be working!
* Profiling a code is to analyze its performance in order to identify how it performs in various situations and the areas where improvement might be needed.
* Profiling enables us to identify the amount of time that a program takes or the amount of memory it uses. 

### timeit Module

* The module `timeit` records the time a **small segment** of your code takes for execution.
* It measures the time elapsed in milliseconds.
* It has both command line as well as callable interfaces.
* It reduces the impact of startup or shutdown costs on the time calculation by executing the code repeatedly. It displays the minimum amount of time it took to run the piece of code.


The syntax when you use `timeit` as a function is:

```python
   timeit.timeit(code_statement, setup, timer, number)
```

where:
 
- `code_statement`: Code statement (as string) for which you want to measure the execution time. The default value is "pass".
- `setup`: Setup details that need to be executed before `code_statement`. The default value is "pass."
- `timer`: Timer value, `timeit()` already has a default value set, and we can ignore it.
- `number`: The number of times `code_statement` will execute. The default value is 1000000.

In [ ]:
import numpy as np
import timeit

In [ ]:
a = np.arange(1000)

%timeit a ** 2
%timeit a ** 2.1
%timeit a * a

In [ ]:
subStrings=['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']

def simpleString(subStrings):
    finalString = ''
    for part in subStrings:
        finalString += part
    return finalString
 
def formatString(subStrings):
    finalString = "%s%s%s%s%s%s%s" % (subStrings[0], subStrings[1],
    subStrings[2], subStrings[3],
    subStrings[4], subStrings[5],
    subStrings[6])
    return finalString
 
def joinString(subStrings):
    return ''.join(subStrings)

print('joinString()   Time : ' + \
      str(timeit.timeit('joinString(subStrings)', 
                        setup='from __main__ import joinString, subStrings')))
print('formatString() Time : '+ \
      str(timeit.timeit('formatString(subStrings)', 
                        setup='from __main__ import formatString, subStrings')))
print('simpleString() Time : ' + \
      str(timeit.timeit('simpleString(subStrings)', 
                        setup='from __main__ import simpleString, subStrings')))

### cprofile Module

* Describe how often and how long various parts of Python code are executed



```python
import cProfile
import pstats
import io

pr = cProfile.Profile()
pr.enable()       # Start collecting profiling data.
# ... do something ...
pr.disable()      # Stop collecting profiling data.
s = io.StringIO()
sortby = 'cumulative'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())
```

        python -m cProfile [-o output_file] [-s sort_order] myscript.py 

#### Example

In [ ]:
%%writefile  write_sorted.py
"""
   Sorting a large, randomly generated string and writing it to disk
"""
import random

def write_sorted_letters(nb_letters=10**7):
    random_string = ''
    for i in range(nb_letters):
        random_string += random.choice('abcdefghijklmnopqrstuvwxyz')
    sorted_string = sorted(random_string)

    with open("sorted_text.txt", "w") as sorted_text:
         for character in sorted_string:
             sorted_text.write(character)

if __name__ == '__main__':
   write_sorted_letters()

In [ ]:
%run -t write_sorted.py

From the command line:

```python
   python -m cProfile -s tottime write_sorted.py
```

## <font color='red'>Import Statement Overhead</font>

* **import** statements can be executed just about anywhere
* For code readbility, it is preferable to include **import** statements on top of the file only.
* May need to place them inside functions to restrict their visibility and/or reduce initial startup time
* Repeatedly executing an **import** statement can seriously affect performance.

To check unused imports in a Python file, consider using <a href="http://pylint.pycqa.org/en/latest/">Pylint</a>.

Import statement inside function:

In [ ]:
def doit_inside():
    import numpy as np
    a = np.array(range(200))

Import statement outside function:

In [ ]:
import numpy as np
def doit_outside():
    a = np.array(range(200))

In [ ]:
t1 = timeit.Timer(setup='from __main__ import doit_inside', stmt='doit_inside()')
t1.timeit()

In [ ]:
t2 = timeit.Timer(setup='from __main__ import doit_outside', stmt='doit_outside()')
t2.timeit()

* For a function that needs to import modules, introduce conditional statements (as far as possible) so that the **import** statements are executed once.

```python
email = None

def parse_email():
    global email
    if email is None:
        import email
    ...
```

## <font color='red'>Functions</font>
* Function call overhead in Python is relatively high.
* Use built-in functions and libraries if they exist instead of creating your own.
* Functions should handle data aggregate.

In [ ]:
import time
x = 0
def doit1(i):
    global x
    x = x + i

t = time.time()
list = range(1000000)
for i in list:
    doit1(i)

print("Time for doit1: %.4f" % (time.time()-t))

In [ ]:
x = 0
def doit2(list):
    global x
    for i in list:
        x = x + i

t = time.time()
list = range(1000000)
doit2(list)

print("Time for doit2: %.4f" % (time.time()-t))

## <font color='red'>Loop Optimization</font>

*  for **loop** can add a substantial amount of the overhead

#### List Comprehension

```python
# Avoid
newlist = []
for word in oldlist:
    newlist.append(word.upper())
    
# Better
newlist = map(str.upper, oldlist)
newlist = [s.upper() for s in oldlist]
```

#### Initializing Dictionary Elements

```python
# Bad
wdict = {}
for word in words:
    if word not in wdict:
        wdict[word] = 0
    wdict[word] += 1

# Better
wdict = {}
for word in words:
    try:
        wdict[word] += 1
    except KeyError:
        wdict[word] = 1
```

### Examples

In [ ]:
import timeit
import itertools

def updateZips(newZipcodes, Zipcodes):
    for zipcode in newZipcodes:
        Zipcodes.append(zipcode.strip())

def updateZipsWithMap(newZipcodes, Zipcodes):
    Zipcodes += map(str.strip, newZipcodes)

def updateZipsWithListCom(newZipcodes, Zipcodes):
    Zipcodes += [iter.strip() for iter in newZipcodes]

def updateZipsWithGenExp(newZipcodes, Zipcodes):
    return itertools.chain(Zipcodes, (iter.strip() for iter in newZipcodes))

In [ ]:
newZipcodes = ['  131313 ',' 242424   ',' 212121 ','  323232','342312  ',' 565656 ']

Zipcodes = ['121212','232323','434334']
print('updateZips() Time            : ' + \
      str(timeit.timeit('updateZips(newZipcodes, Zipcodes)', 
                        setup='from __main__ import updateZips, newZipcodes, Zipcodes')))

Zipcodes = ['121212','232323','434334']
print('updateZipsWithMap() Time     : ' + \
      str(timeit.timeit('updateZipsWithMap(newZipcodes, Zipcodes)', 
                        setup='from __main__ import updateZipsWithMap, newZipcodes, Zipcodes')))

Zipcodes = ['121212','232323','434334']
print('updateZipsWithListCom() Time : ' + \
      str(timeit.timeit('updateZipsWithListCom(newZipcodes, Zipcodes)', 
                        setup='from __main__ import updateZipsWithListCom, newZipcodes, Zipcodes')))

Zipcodes = ['121212','232323','434334']
print('updateZipsWithGenExp() Time  : ' + \
      str(timeit.timeit('updateZipsWithGenExp(newZipcodes, Zipcodes)', 
                        setup='from __main__ import updateZipsWithGenExp, newZipcodes, Zipcodes')))

## <font color='red'>String Manipulation</font>

#### Looking to create a string

```python
# Avoid
s = ""
for x in list:
    s += some_function(x)

# Better
slist = [some_function(elt) for elt in somelist]
s = "".join(slist)
```

#### Use string formatting

```python
# Avoid
out = "<html>" + head + prologue + query + tail + "</html>"

# Better
out = "<html>%s%s%s%s</html>" % (head, prologue, query, tail)

#or

out = "<html>%(head)s%(prologue)s%(query)s%(tail)s</html>" % locals()

#or

out = "<html>{}{}{}{}</html>".format(head, prologue, query, tail)
```

#### Exercise

Time each of the for code segments below

In [ ]:
# slow O(n^2) 
s = 'Advance Python Bootcamp at Goddard Space Flight Center'
slist = '' 
for i in s: 
    slist = slist + i 
print(slist) 
      
# string concatenation (idiomatic and fast O(n)) 
s = 'Advance Python Bootcamp at Goddard Space Flight Center'
slist = ''.join([i for i in s]) 
print(slist) 
  
# slow 
p = ' Python'
b = ' Bootcamp'
g = ' Goddard'
f = ' Flight'
c = ' Center'
s = 'Advance'+ p + b +' at' + g + ' Space' + f + c
print(s) 
  
# fast 
v = 'AND'
s = 'geeks %s geeks' % v 
s = 'Advance %s%s at %s Space %s%s' %(p,b,g,f,c)
print(s) 

## <font color='red'>Numerical Codes</font>

#### List vs Numpy Arrays

*  Use Numpy arrays instead of list

#### Loops vs Vectorization

* Avoid for loops using numpy arrays
* Use array slicing or masks

###  In Place Arithmetic

In [ ]:
import numpy as np
n = 100000000
a = np.zeros(n)
%timeit global a ; a = 0*a

a = np.zeros(n)
%timeit global a ; a *= 0

### Use Views and not Copy 

* Copying big arrays is as costly as making simple numerical operations on them

In [ ]:
import numpy as np
n = 100000000
a = np.zeros(n)
%timeit a.copy()

a = np.zeros(n)
%timeit a + 1

### Cache Effects

* Memory access is cheaper when it is grouped
* Accessing a big array in a continuous way is much faster than random access
* Smaller strides are faster

In [ ]:
import numpy as np
n = 20000
c = np.zeros((n, n), order='C')

%timeit c.sum(axis=0)
%timeit c.sum(axis=1)

print(c.strides)

This is the reason why Fortran ordering or C ordering may make a big difference on operations:

In [ ]:
a = np.random.rand(200, 2**18)

b = np.random.rand(200, 2**18)
%timeit np.dot(b, a.T)

c = np.ascontiguousarray(a.T)
%timeit np.dot(b, c)

# Note that copying the data to work around this effect 
# may not be worth it:
%timeit c = np.ascontiguousarray(a.T)

#### <font color='blue'>Memoization</font>
- Some function values are computed over and over again, wasting time and computational resources.
- We want to be able to compute each value once and reuse it as needed.
- We use the `lru_cache` decorator of the module `functools`.
- It enables your function to store already computed values and reuse them when needed again.

In [ ]:
def fibonacci(n):
    if n < 2:
       return 1
    return fibonacci(n-1) + fibonacci(n-2)

In [ ]:
import functools

@functools.lru_cache(maxsize=128)
def fibonacci_cach(n):
    if n < 2:
       return 1
    return fibonacci_cach(n-1) + fibonacci_cach(n-2)

In [ ]:
n = 35

In [ ]:
time_fibonacci_reg = %timeit -o fibonacci(n)

In [ ]:
time_fibonacci_cach = %timeit -o fibonacci_cach(n)

In [ ]:
print(time_fibonacci_reg.best/time_fibonacci_cach.best)

## <font color='red'>Memory Optimization</font>

* Limit the number of variables
* If possible avoid using the **global** keyword. Python is faster retrieving local variables.
* Set variables (especially large arrays) to **None** when no longer used.